# Pforzheim Smart City - Sensor-Standort-Analyse 
URL: [https://www.smartcity-pforzheim.de/index.html]

# Start der Sensor-Standort-Analyse

In [31]:
import pandas as pd
import json
import re


PATH = './data/devices.json'

# Lade die JSON-Datei
file_path = PATH
with open(file_path, "r", encoding="utf-8") as file:
    data = json.load(file)

# Extrahiere die relevanten Daten aus der JSON-Struktur in ein DataFrame
df = pd.json_normalize(
    data, record_path=["values"], meta=["description", "deviceId", "model"]
)

# Zeige das DataFrame an
df.head(5)

,origin,reference,timestamp,value,alphanumericValue,datapoint.datapoint,datapoint.description,datapoint.refNr,datapoint.refType,datapoint.unit,description,deviceId,model
0,"To get information about origin, please fill i...",474f5350fb070820,2024-12-16T11:46:05.000+01:00,-107.0000,To get information about alphanumeric value e....,0-0:0.0.12,RSSI Uplink,2544134,Geräte,dBm,"Habermehlstraße 0, FB070820, rechts",474f5350fb070820,Fleximodo Sensor 2.0
1,"To get information about origin, please fill i...",474f5350fb070820,2024-12-16T11:46:05.000+01:00,2.0000,To get information about alphanumeric value e....,0-0:0.0.2,Rohdaten,2544134,Geräte,,"Habermehlstraße 0, FB070820, rechts",474f5350fb070820,Fleximodo Sensor 2.0
2,"To get information about origin, please fill i...",474f5350fb070820,2024-12-16T11:46:05.000+01:00,0.0000,To get information about alphanumeric value e....,0-0:96.54.0,Error Flags,2544134,Geräte,-,"Habermehlstraße 0, FB070820, rechts",474f5350fb070820,Fleximodo Sensor 2.0
3,"To get information about origin, please fill i...",474f5350fb070820,2024-12-16T11:46:05.000+01:00,88.9000,To get information about alphanumeric value e....,0-0:96.6.1,Batteriestatus,2544134,Geräte,%,"Habermehlstraße 0, FB070820, rechts",474f5350fb070820,Fleximodo Sensor 2.0
4,"To get information about origin, please fill i...",474f5350fb070820,2024-12-16T11:46:05.000+01:00,0.0000,To get information about alphanumeric value e....,98-0:1.0.0,Status,2544134,Geräte,None,"Habermehlstraße 0, FB070820, rechts",474f5350fb070820,Fleximodo Sensor 2.0


In [32]:
descriptions_df = df[["description","deviceId"]
].drop_duplicates()
descriptions_df.head(25)

,description,deviceId
0,"Habermehlstraße 0, FB070820, rechts",474f5350fb070820
9,"Deimlingstraße 23, FB071642, LS 2 links",474f5350fb071642
18,BD-WS-mR-9\r\nRegensensor S/N: B240612050,0004a30b010a64eb
32,"Kiehnlestraße 26, FB070821, links",474f5350fb070821
41,"Deimlingstraße 23, FB071641, LS 1 rechts",474f5350fb071641
50,"Kallhardtstraße 62, FB071636, LS 2 links",474f5350fb071636
59,Nr. 2,24e124136c452759
68,"FB070031, Simmlerstraße 3",474f5350fb070031
77,Nr. 11,24E124136E094322
85,BD-WS-mR-4\r\nRegensensor S/N: B240612039,0004a30b0109f170


# Aufteilen der Daten in Spalte "description"

In [33]:
descriptions_df = pd.DataFrame(descriptions_df, columns=["description", "deviceId"])

# Spalte anhand des Kommas splitten
df_split = df["description"].str.split(",", expand=True)

# Neue Spalten in den DataFrame einfügen
descriptions_df[["Straße", "Bezeichner", "Zusatzinformation"]] = df_split

# DataFrame anzeigen
descriptions_df.head(5)

,description,deviceId,Straße,Bezeichner,Zusatzinformation
0,"Habermehlstraße 0, FB070820, rechts",474f5350fb070820,Habermehlstraße 0,FB070820,rechts
9,"Deimlingstraße 23, FB071642, LS 2 links",474f5350fb071642,Deimlingstraße 23,FB071642,LS 2 links
18,BD-WS-mR-9\r\nRegensensor S/N: B240612050,0004a30b010a64eb,BD-WS-mR-9\r\nRegensensor S/N: B240612050,None,None
32,"Kiehnlestraße 26, FB070821, links",474f5350fb070821,Kiehnlestraße 26,FB070821,links
41,"Deimlingstraße 23, FB071641, LS 1 rechts",474f5350fb071641,Deimlingstraße 23,FB071641,LS 1 rechts


In [34]:
# Zeilen entfernen, bei denen die Straße nur "Nr. X" enthält
df_clean = descriptions_df[
    ~descriptions_df["Straße"].str.contains(r"^Nr\.\s*\d+$", na=False)
]
df_clean[0:25]

,description,deviceId,Straße,Bezeichner,Zusatzinformation
0,"Habermehlstraße 0, FB070820, rechts",474f5350fb070820,Habermehlstraße 0,FB070820,rechts
9,"Deimlingstraße 23, FB071642, LS 2 links",474f5350fb071642,Deimlingstraße 23,FB071642,LS 2 links
18,BD-WS-mR-9\r\nRegensensor S/N: B240612050,0004a30b010a64eb,BD-WS-mR-9\r\nRegensensor S/N: B240612050,None,None
32,"Kiehnlestraße 26, FB070821, links",474f5350fb070821,Kiehnlestraße 26,FB070821,links
41,"Deimlingstraße 23, FB071641, LS 1 rechts",474f5350fb071641,Deimlingstraße 23,FB071641,LS 1 rechts
50,"Kallhardtstraße 62, FB071636, LS 2 links",474f5350fb071636,Kallhardtstraße 62,FB071636,LS 2 links
68,"FB070031, Simmlerstraße 3",474f5350fb070031,FB070031,Simmlerstraße 3,None
85,BD-WS-mR-4\r\nRegensensor S/N: B240612039,0004a30b0109f170,BD-WS-mR-4\r\nRegensensor S/N: B240612039,None,None
99,Nr. 3 - HS,3233343072308b16,Nr. 3 - HS,None,None
125,"Melanchthonstraße 7, FB071633, LS 2 rechts",474f5350fb071633,Melanchthonstraße 7,FB071633,LS 2 rechts


# Daten bereinigen die keine Adresse haben
## Adresse finden und werte tauschen
Hier werden die Werte in den Spalten für Straße und Bezeichner korrigiert

In [35]:
search_pattern = r"(straße|platz|gasse|allee|weg|park|haus|parkplatz)"

# Funktion, die prüft, ob ein Wert eine Adresse ist
def ist_adresse(wert):
    if pd.isna(wert):
        return False
    # Erkennung von Adressen Werten
    else:
        return bool(re.search(search_pattern, wert, re.IGNORECASE))


# Werte prüfen und ggf. tauschen
def prüfe_und_tausche(row):
    if ist_adresse(row["Straße"]):
        return pd.Series({"Straße": row["Straße"], "Bezeichner": row["Bezeichner"]})

    elif ist_adresse(row["Straße"]) and ist_adresse(row["Bezeichner"]):
        return pd.Series({"Straße": row["Bezeichner"], "Bezeichner": row["Straße"]})

    elif not ist_adresse(row["Straße"]) and ist_adresse(row["Bezeichner"]):
        return pd.Series({"Straße": row["Bezeichner"], "Bezeichner": row["Straße"]})

    else:
        return pd.Series({"Straße": row["Bezeichner"], "Bezeichner": row["Straße"]})


# Anwenden der Funktion auf den DataFrame
df_clean[["Straße", "Bezeichner"]] = df_clean.apply(prüfe_und_tausche, axis=1)

# Ergebnis anzeigen
df_clean[0:20]

/tmp/ipykernel_714170/3728899649.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[["Straße", "Bezeichner"]] = df_clean.apply(prüfe_und_tausche, axis=1)


,description,deviceId,Straße,Bezeichner,Zusatzinformation
0,"Habermehlstraße 0, FB070820, rechts",474f5350fb070820,Habermehlstraße 0,FB070820,rechts
9,"Deimlingstraße 23, FB071642, LS 2 links",474f5350fb071642,Deimlingstraße 23,FB071642,LS 2 links
18,BD-WS-mR-9\r\nRegensensor S/N: B240612050,0004a30b010a64eb,None,BD-WS-mR-9\r\nRegensensor S/N: B240612050,None
32,"Kiehnlestraße 26, FB070821, links",474f5350fb070821,Kiehnlestraße 26,FB070821,links
41,"Deimlingstraße 23, FB071641, LS 1 rechts",474f5350fb071641,Deimlingstraße 23,FB071641,LS 1 rechts
50,"Kallhardtstraße 62, FB071636, LS 2 links",474f5350fb071636,Kallhardtstraße 62,FB071636,LS 2 links
68,"FB070031, Simmlerstraße 3",474f5350fb070031,Simmlerstraße 3,FB070031,None
85,BD-WS-mR-4\r\nRegensensor S/N: B240612039,0004a30b0109f170,None,BD-WS-mR-4\r\nRegensensor S/N: B240612039,None
99,Nr. 3 - HS,3233343072308b16,None,Nr. 3 - HS,None
125,"Melanchthonstraße 7, FB071633, LS 2 rechts",474f5350fb071633,Melanchthonstraße 7,FB071633,LS 2 rechts


In [36]:

search_pattern = r"(FB)"

# Funktion, die prüft, ob ein Wert ein Bezeichner ist
def ist_bezeichner(wert):
    if pd.isna(wert):
        return False
    # Erkennung von Bezeichner Werten
    else:
        return bool(re.search(search_pattern, wert, re.IGNORECASE))


# Werte prüfen und ggf. tauschen
def prüfe_und_tausche(row):
    if ist_bezeichner(row["Bezeichner"]):
        return pd.Series(
            {
                "Bezeichner": row["Bezeichner"],
                "Zusatzinformation": row["Zusatzinformation"],
            }
        )
    elif ist_bezeichner(row["Bezeichner"]) and ist_bezeichner(row["Zusatzinformation"]):
        return pd.Series(
            {
                "Bezeichner": row["Zusatzinformation"],
                "Zusatzinformation": row["Bezeichner"],
            }
        )
    elif not ist_bezeichner(row["Straße"]) and ist_bezeichner(row["Zusatzinformation"]):
        return pd.Series(
            {
                "Bezeichner": row["Zusatzinformation"],
                "Zusatzinformation": row["Bezeichner"],
            }
        )
    else:
        return pd.Series(
            {
                "Bezeichner": row["Bezeichner"],
                "Zusatzinformation": row["Zusatzinformation"],
            }
        )


# Anwenden der Funktion auf den DataFrame
df_clean[["Bezeichner", "Zusatzinformation"]] = df_clean.apply(prüfe_und_tausche, axis=1)

# Ergebnis anzeigen
df_clean[0:20]

/tmp/ipykernel_714170/412896812.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[["Bezeichner", "Zusatzinformation"]] = df_clean.apply(prüfe_und_tausche, axis=1)


,description,deviceId,Straße,Bezeichner,Zusatzinformation
0,"Habermehlstraße 0, FB070820, rechts",474f5350fb070820,Habermehlstraße 0,FB070820,rechts
9,"Deimlingstraße 23, FB071642, LS 2 links",474f5350fb071642,Deimlingstraße 23,FB071642,LS 2 links
18,BD-WS-mR-9\r\nRegensensor S/N: B240612050,0004a30b010a64eb,None,BD-WS-mR-9\r\nRegensensor S/N: B240612050,None
32,"Kiehnlestraße 26, FB070821, links",474f5350fb070821,Kiehnlestraße 26,FB070821,links
41,"Deimlingstraße 23, FB071641, LS 1 rechts",474f5350fb071641,Deimlingstraße 23,FB071641,LS 1 rechts
50,"Kallhardtstraße 62, FB071636, LS 2 links",474f5350fb071636,Kallhardtstraße 62,FB071636,LS 2 links
68,"FB070031, Simmlerstraße 3",474f5350fb070031,Simmlerstraße 3,FB070031,None
85,BD-WS-mR-4\r\nRegensensor S/N: B240612039,0004a30b0109f170,None,BD-WS-mR-4\r\nRegensensor S/N: B240612039,None
99,Nr. 3 - HS,3233343072308b16,None,Nr. 3 - HS,None
125,"Melanchthonstraße 7, FB071633, LS 2 rechts",474f5350fb071633,Melanchthonstraße 7,FB071633,LS 2 rechts


In [37]:
# Drop alle Werte mit None in Straße
df_clean = df_clean.dropna(subset=["Straße"])
# Drop alle doppleten Werte
df_clean = df_clean.drop_duplicates(subset=["Straße"])

df_clean["Straße"][0:20]

0                     Habermehlstraße 0
9                     Deimlingstraße 23
32                     Kiehnlestraße 26
50                   Kallhardtstraße 62
68                      Simmlerstraße 3
125                 Melanchthonstraße 7
134                Kronprinzenstraße 33
173                     Goethestraße 15
193    Östliche Karl-Friedrich-Straße 7
213                  Osterfeldstraße 37
233                           Messplatz
268                       Inselstraße 2
293                       Fondelystraße
302               Robert-Bauer-Straße 8
320                      Frankstraße 71
350               Am Waisenhausplatz 12
359               Hohenzollernstraße 19
429                    Belfortstraße 46
447                       Jahnstraße 38
527                      Salierstraße 7
Name: Straße, dtype: object

In [38]:
import pandas as pd
import folium
from geopy.geocoders import Nominatim # type: ignore
from geopy.exc import GeocoderTimedOut # type: ignore

# DataFrame erstellen
df = pd.DataFrame(df_clean["Straße"])

# Geolokalisierung mit geopy
geolocator = Nominatim(user_agent="myGeocoder")


def geocode_address(address):
    try:
        location = geolocator.geocode(f"{address}, Pforzheim, Deutschland")
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except GeocoderTimedOut:
        return None, None


# Adressen in Breiten- und Längengrade umwandeln
df["Latitude"], df["Longitude"] = zip(*df["Straße"].apply(geocode_address))

# Zeilen mit fehlenden Geokoordinaten entfernen
df_cleaned = df.dropna(subset=["Latitude", "Longitude"])

# Folium-Karte erstellen
map_pforzheim = folium.Map(
    location=[48.8921, 8.6946], zoom_start=13
)  # Pforzheim Koordinaten als Zentrum

# Marker für jede Adresse hinzufügen
for _, row in df_cleaned.iterrows():
    folium.Marker(
        location=[row["Latitude"], row["Longitude"]],
        popup=row["Straße"],
    ).add_to(map_pforzheim)

# Karte speichern und anzeigen
map_pforzheim.save(
    "./sensor_location_map/sensor_location_map.html"
)
print("Die Karte wurde gespeichert.")

Die Karte wurde gespeichert.


# ToDo: Unvollständige Daten für Qualitätssicherung finden

In [39]:
# Vergleich zwischen Original und bereinigtem Datensatz

df_error = df[~df.index.isin(df_clean.index)]

# df_error.head(5)